In [1]:
import numpy as np
import pandas as pd
from copy import deepcopy
import os
import json

In [2]:
folder_path = "../../LongitudinalDataSet/"

# Get new Phase II

## Select Pre&Post from old table

In [3]:
df_aggregated_phaseII_old = pd.read_csv(folder_path + "aggregated_phaseII.csv")

C:\ProgramPool\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (382,395,399,412,416,429,433,446,450,463,466,467,468,471,472,475,476,477,479,484,491,499,502,506,508,509,510,511,512,515,516,519,520,521,523,528,535,543,546,550,552,553,554,555,556,559,560,563,564,565,567,572,579,587,590,594,596,597,598,599,600,603,604,607,608,609,611,616,623,631,634,638,640,641,642,643,644,647,648,651,652,653,655,660,667,675,678,682,684,685) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
cols_from_old = df_aggregated_phaseII_old.columns.tolist()[0:1] + df_aggregated_phaseII_old.columns.tolist()[2:3] + df_aggregated_phaseII_old.columns.tolist()[-36:]

In [5]:
pre_post = df_aggregated_phaseII_old[cols_from_old]

In [7]:
pre_post = pd.read_csv(folder_path + "survey_prepost_phaseI.csv")

## Select EMA as the base

In [34]:
ema = pd.read_csv(folder_path + "aggregated_ema_phaseI.csv")

C:\ProgramPool\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (61,82,83) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [35]:
ema.shape

(18232, 93)

In [36]:
ema_baseline = ema.merge(pre_post, on=["PID"],how="left")

# Select sensor

In [16]:
wks = ["wkdy", "wkend"]
eps = ["mo","af","ev","ni",""]

In [17]:
dfdict_ep_wk_phaseII = {}
for ep in eps:
    dfdict_ep_wk_phaseII[ep] = {}
    for wk in wks:
        dfdict_ep_wk_phaseII[ep][wk] = {}
        name = ",_".join(["dis_merge", ep, wk, "day"]) + ".csv"
        print(name)
        dfdict_ep_wk_phaseII[ep][wk] = pd.read_csv(folder_path + "BehaviorFeatures_merge_day_PhaseI/" + name)

dis_merge,_mo,_wkdy,_day.csv
dis_merge,_mo,_wkend,_day.csv


C:\ProgramPool\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


dis_merge,_af,_wkdy,_day.csv
dis_merge,_af,_wkend,_day.csv
dis_merge,_ev,_wkdy,_day.csv
dis_merge,_ev,_wkend,_day.csv
dis_merge,_ni,_wkdy,_day.csv
dis_merge,_ni,_wkend,_day.csv
dis_merge,_,_wkdy,_day.csv
dis_merge,_,_wkend,_day.csv


In [18]:
for ep in eps:
    for wk in wks:
        print(dfdict_ep_wk_phaseII[ep][wk].isna().sum().sum()/ float(dfdict_ep_wk_phaseII[ep][wk].shape[0] * dfdict_ep_wk_phaseII[ep][wk].shape[1]))

0.31480075723496775
0.4013582225387781
0.3443454947018543
0.3718571937321937
0.33557161243564754
0.3552132795188351
0.4183711482231219
0.39120469294080407
0.20910923364322487
0.22540459797404241


In [19]:
for ep in eps:
    dfdict_ep_wk_phaseII[ep]["wk"] = pd.concat([dfdict_ep_wk_phaseII[ep]["wkdy"], dfdict_ep_wk_phaseII[ep]["wkend"]], axis = 0)

In [22]:
device_pid_mapping = pd.read_csv(folder_path + "PID-device_PhaseI.csv", dtype=str)
dict_device_pid = {}
for i, row in device_pid_mapping.iterrows():
    for x in list(row[device_pid_mapping.columns[1:]]):
        if (type(x) is str):
            dict_device_pid[x.split("-")[-1]] = row["PID"]

In [11]:
device_pid_mapping = pd.read_csv(folder_path + "PID-device_PhaseII.csv", dtype=str)
dict_device_pid = {}
for i, row in device_pid_mapping.iterrows():
    for x in list(row[device_pid_mapping.columns[2:6]]):
        if (type(x) is str):
            dict_device_pid[x] = row["PID"]

In [24]:
def device_to_pid(device_id):
    if ("-" in device_id):
        x = device_id.split("-")[-1]
    if (len(x) == 12):
        if (x in dict_device_pid):
            return dict_device_pid[x]
        else:
            return "NOMATCH"
    else:
        return "IDWRONG"

In [25]:
dict_epoch = {
    "":"allday",
    "mo":"morning",
    "af":"afternoon",
    "ev":"evening",
    "ni":"night"
}

In [26]:
for ep in eps:
    cols = dfdict_ep_wk_phaseII[ep]["wk"].columns[2:]
    rename_cols = {col : col + "_" + dict_epoch[ep] for col in cols}
    dfdict_ep_wk_phaseII[ep]["wk"].rename(rename_cols, axis = 1,inplace = True)
    dfdict_ep_wk_phaseII[ep]["wk"]["PID"] = dfdict_ep_wk_phaseII[ep]["wk"]["device_id"].apply(lambda x : device_to_pid(x))
    print("remove unique id: ", len(set(list(dfdict_ep_wk_phaseII[ep]["wk"]["device_id"][dfdict_ep_wk_phaseII[ep]["wk"]["PID"] == "NOMATCH"]))))
    dfdict_ep_wk_phaseII[ep]["wk"].drop("device_id", axis = 1, inplace = True)
    dfdict_ep_wk_phaseII[ep]["wk"] = dfdict_ep_wk_phaseII[ep]["wk"][dfdict_ep_wk_phaseII[ep]["wk"]["PID"] != "NOMATCH"]
    cols = list(dfdict_ep_wk_phaseII[ep]["wk"].columns)
    dfdict_ep_wk_phaseII[ep]["wk"] = dfdict_ep_wk_phaseII[ep]["wk"][[cols[0],cols[-1]] + cols[1:-1]]

remove unique id:  0
remove unique id:  0
remove unique id:  0
remove unique id:  0
remove unique id:  0


In [27]:
# In EMA and baseline, Remove PID without any sensor data 
pid_with_sensor = []
for ep in eps:
    pid_with_sensor += dfdict_ep_wk_phaseII[ep]["wk"]["PID"].tolist()
pid_with_sensor = list(set(pid_with_sensor))
ema_baseline["PID"] = ema_baseline["PID"].astype(str)
ema_baseline = ema_baseline[ema_baseline["PID"].apply(lambda x : str(x) in pid_with_sensor)]

In [28]:
df_sensor = deepcopy(dfdict_ep_wk_phaseII[eps[0]]["wk"])
for ep in eps[1:]:
    df_sensor = df_sensor.merge(dfdict_ep_wk_phaseII[ep]["wk"], on = ["PID","date"], how="left")
cols = list(df_sensor.columns[2:])
len_feature = len(cols)
new_idx = []
for i in range(int(len_feature / 5)):
    new_idx += list(np.array(range(5))*int(len_feature/5) + i)
cols_new = list(df_sensor.columns[:2]) + list(np.array(cols)[new_idx])
df_sensor = df_sensor[cols_new]

In [31]:
df_sensor.shape

(15264, 2332)

In [32]:
df_aggregated.shape

(15264, 2455)

In [41]:
pd.to_datetime("2017-05-03") - pd.to_datetime("2017-01-18")

Timedelta('105 days 00:00:00')

In [ ]:
ema_baseline

In [33]:
ema_baseline.shape

(15264, 125)

In [29]:
df_aggregated = pd.merge(ema_baseline, df_sensor, on = ["PID","date"], how="left")

In [30]:
df_aggregated.to_csv(folder_path + "aggregated_all_phaseI.csv", index = False)